# Qiskit Aer Backends

* *Author*: Christopher J. Wood (cjwood@us.ibm.com)
* *Last Updated*: December 10, 2018

In [1]:
import numpy as np

# Import Terra
import qiskit
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister

# Import Aer backends
from qiskit_aer import Aer

This notebook shows how to import Qiskit Aer simulators and use them as backends for Qiskit Terra.

Currently Qiskit Aer includes three simulator backends:
* **QasmSimulator**: allows ideal and noisy multi-shot execution of qiskit circuits and returns counts or memory
* **StatevectorSimulator**: allows ideal single-shot execution of qiskit circuits and returns the final statevector of the simulator after application
* **UnitarySimulator**: allows ideal single-shot execution of qiskit circuits and returns the final unitary matrix of the circuit itself. Note that the circuit cannot contain measure or reset operations for this backend

These backends are found in the `Aer` provider with the names `qasm_simulstor`, `statevector_simulator` and `unitary_simulator` respectively

In [2]:
# List Aer backends
Aer.backends()

[<QasmSimulator('qasm_simulator') from AerProvider()>,
 <StatevectorSimulator('statevector_simulator') from AerProvider()>,
 <UnitarySimulator('unitary_simulator') from AerProvider()>]

### Example circuits

We will consider execution of the following example circuits which prepare an N-qubit entangled GHZ state either with or without final measurement at the end

In [3]:
# Test circuits

def ghz_circuit_nomeas(num_qubits):
    qr = qiskit.QuantumRegister(num_qubits)
    circ = qiskit.QuantumCircuit(qr)
    circ.h(qr[0])
    if num_qubits > 1:
        for j in range(num_qubits - 1):
            circ.cx(qr[j], qr[j + 1])
    return circ

def ghz_circuit_meas(num_qubits):
    qr = qiskit.QuantumRegister(num_qubits)
    cr = qiskit.ClassicalRegister(num_qubits)
    circ = qiskit.QuantumCircuit(qr, cr)
    circ.h(qr[0])
    if num_qubits > 1:
        for j in range(num_qubits - 1):
            circ.cx(qr[j], qr[j + 1])
    circ.barrier(qr)
    circ.measure(qr, cr)
    return circ

## Qasm Simulator

This Qasm simulator backend is designed to mimic an actual device. It executes a qiskit circuit and returns a count dictionary containing the final values of any classical registers in the circuit. The circuit may contain gates, measure, reset, or conditionals, and other advanced simulator options that will be discussed in another notebook.

In [4]:
circ = ghz_circuit_meas(4)
backend = Aer.get_backend('qasm_simulator')
result = qiskit.execute(circ, backend).result()
result.get_counts(circ)

{'0000': 530, '1111': 494}

## Statevector simulator

The statevector simulator returns the final statevector after applying a quantum circuit to an initial state. By default this initial state is all qubits in $|0\rangle$.

The following circuit shows the final statevector after preparing a 4-qubit GHZ state

In [5]:
circ = ghz_circuit_nomeas(4)
backend = Aer.get_backend('statevector_simulator')
result = qiskit.execute(circ, backend).result()
result.get_statevector(circ)

array([0.70710678+0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
       0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
       0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
       0.        +0.j, 0.        +0.j, 0.        +0.j, 0.70710678+0.j])

Note that we used a circuit containing no measure operations. If a circuit contains measure or reset options the final statevector will be a conditional state after collapse from the measurement. For example if we consider the 4-qubit GHZ circuit with measurement the final state will be either $|0000\rangle$ *or* $|1111\rangle$

In [6]:
circ = ghz_circuit_meas(4)
backend = Aer.get_backend('statevector_simulator')
result = qiskit.execute(circ, backend).result()
result.get_statevector(circ)

array([0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j])

## Unitary Simulator

The unitary simulator comutes the unitary matrix for a given quantum circuit by applying each gate to an initial identity matrix. The circuit cannot contain measure or reset operations.

The following example shows the unitary corresponding to a Bell-state preparation circuit

In [7]:
circ = ghz_circuit_nomeas(2)
backend = Aer.get_backend('unitary_simulator')
result = qiskit.execute(circ, backend).result()
result.get_unitary(circ)

array([[ 0.70710678+0.j,  0.70710678+0.j,  0.        +0.j,
         0.        +0.j],
       [ 0.        +0.j,  0.        +0.j,  0.70710678+0.j,
        -0.70710678+0.j],
       [ 0.        +0.j,  0.        +0.j,  0.70710678+0.j,
         0.70710678+0.j],
       [ 0.70710678+0.j, -0.70710678+0.j,  0.        +0.j,
         0.        +0.j]])